# 삼성전자 종기원 시각 심화

- **Instructor**: Jongwoo Lim / Jiun Bae
- **Email**: [jlim@hanyang.ac.kr](mailto:jlim@hanyang.ac.kr) / [jiun.maydev@gmail.com](mailto:jiun.maydev@gmail.com)

## MNIST Example

In this example you will practice a simple neural network written by [TensorFlow](https://www.tensorflow.org) and [Keras](https://keras.io), using the basically used handwritten digits data set, [MNIST Dataset](http://yann.lecun.com/exdb/mnist). The goals of this example are as follows:

- Learn basically how to **write and use code**(*TensorFlow (Keras)*).
- Understand **Neural Networks** and how they work.

And this example also is written in [IPython Notebook](https://ipython.org/notebook.html), an interactive computational environment, in which you can run code directly.

### Environments

In this assignment, we assume the follows environments. 
The [Python](https://www.python.org) is a programming language that lets you work quickly and integrate systems more effectively. It is widely used in various fields, and also used in machine learning.

The [CUDA®](https://developer.nvidia.com/cuda-zone) Toolkit provides high-performance GPU-accelerated computation. In deep learning, the model takes an age to train without GPU-acceleration. ~~even with the GPU, it still takes a lot of time~~.


- [Python3](https://www.python.org/downloads/) (recommend 3.6 or above)
- [TensorFlow](https://www.tensorflow.or) is famous deep learning platform by Google (recommend 1.13 or above).
- [Keras](https://keras.io) TensorFlow 2.0 or above contain keras as high-level API.


- (Optional) [Anaconda](https://www.anaconda.com/distribution/#download-section), *popular Python Data Science Platform*
- (Optional) [PyTorch](https://pytorch.org) is an open source deep learning platform. (recommend 1.0 or above)
- (Optional) [Jupyter](https://jupyter.org/) (Notebook or Lab)
- (Optional) [CUDA](https://developer.nvidia.com/cuda-downloads) support GPU


Python packages can install by `pip install [package name]` or using **Anaconda** by `conda install [package name]`.

*If you are having trouble installing or something else, please contact TA or jiun.maydev@gmail.com.*

## Load MNIST dataset

Tesnorflow provide mnist dataset as binary archive file [link](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/tutorials/mnist/download/index.md).
In this exampe, we already downloaded datafile in `./data` directory. So just laod dataset from `./data`.

In [ ]:
import numpy as np
from tensorflow.python import keras

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data('mnist.npz')

## (Optional) Visualize

In [ ]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))

In [ ]:
for image, label, _ in zip(train_images, train_labels, range(5)):
    print(label)
    show(image.reshape((28, 28)))

## Preprocessing

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255. We scale these values to a range of 0 to 1 before feeding to the neural network model. For this, we divide the values by 255. It's important that the training set and the testing set are preprocessed in the same way:

In [ ]:
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

train_images = train_images / 255.
test_images = test_images / 255.

In [ ]:
from tensorflow.python.keras.utils import to_categorical
num_classes = 10
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

## Model

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28, 1)),
    keras.layers.Dense(128, activation=keras.activations.sigmoid),
    keras.layers.Dense(10, activation=keras.activations.sigmoid)
])

## Compile

In [ ]:
model.compile(optimizer='sgd',
              loss='mse',
              metrics=['accuracy'])

## Train

In [ ]:
model.fit(train_images, train_labels, epochs=1)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Accuracy:', test_acc)

## Predictions

In [ ]:
predictions = model.predict(test_images)

In [ ]:
for image, prediction, label, _ in (zip(test_images, predictions, test_labels, range(5))):
    print(f'Label: {np.argmax(label)}, Prediction: {np.argmax(prediction)}')
    show((image[:, :, 0] * 255.).astype(np.uint8))

## Model (with ReLU)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=keras.activations.relu),
    keras.layers.Dense(10, activation=keras.activations.sigmoid)
])

## Model (with CNN)

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation=keras.activations.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation=keras.activations.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=keras.activations.relu),
    keras.layers.Dense(10, activation=keras.activations.relu),
])

## Model (with CNN, Softmax)

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation=keras.activations.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation=keras.activations.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=keras.activations.relu),
    keras.layers.Dense(10, activation=keras.activations.softmax),
])

In [ ]:
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## VGG16 

In [ ]:
from tensorflow.python.keras.applications.vgg16 import VGG16

In [ ]:
model = keras.models.Sequential([
    VGG16(include_top=False, weights=None, classes=10, input_shape=(56, 56, 3)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(10, activation='softmax'),
])

### 2x image dataset

In [ ]:
from PIL import Image

double_train_images = np.empty((60000, 56, 56, 3), dtype=np.float32)
double_test_images = np.empty((10000, 56, 56, 3), dtype=np.float32)

for i, image in enumerate(train_images):
    image = np.array(Image.fromarray((image[:, :, 0] * 255.).astype(np.uint8)).resize((56, 56))) / 255.
    double_train_images[i] = np.transpose(np.stack((image, image, image)), (1, 2, 0))

for i, image in enumerate(test_images):
    image = np.array(Image.fromarray((image[:, :, 0] * 255.).astype(np.uint8)).resize((56, 56))) / 255.
    double_test_images[i] = np.transpose(np.stack((image, image, image)), (1, 2, 0))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(double_train_images, train_labels, epochs=1)

In [ ]:
test_loss, test_acc = model.evaluate(double_test_images, test_labels)

print('Accuracy:', test_acc)

## VGG16 with pretrained

In [ ]:
model = keras.models.Sequential([
    VGG16(include_top=False, classes=10, input_shape=(56, 56, 3)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(10, activation='softmax'),
])